In [2]:
from fastapi import FastAPI
import pandas as pd
import numpy as np

In [5]:
df_playtime_genre = pd.read_csv('FeatureEngData/df_playtime_genre.csv',low_memory=False)
df_sentiment_analysis = pd.read_csv('FeatureEngData/df_sentimennt_analysis.csv',low_memory=False)
df_user_for_genre = pd.read_csv('FeatureEngData/df_user_for_genre.csv',low_memory=False)
df_UsersRecommend = pd.read_csv('FeatureEngData/df_users_recommend.csv',low_memory=False)

In [6]:
df_playtime_genre

,genres,release_year,playtime_forever
0,1980s,2015,23930
1,1980s,No data,3951
2,1990s,2011,172945
3,1990s,2012,57473
4,2D,2012,167402
...,...,...,...
789,WorldWarII,No data,43035305
790,Zombies,2015,2728
791,Zombies,No data,78741
792,e-sports,2014,448600


In [7]:
genre = 'Action'

df_genero = df_playtime_genre[df_playtime_genre['genres'].str.lower() == genre.lower() if isinstance(genre, str) else None]

In [8]:
df_genero

,genres,release_year,playtime_forever
12,Action,1983,3473
13,Action,1984,384
14,Action,1988,16001
15,Action,1989,607
16,Action,1990,18335
17,Action,1991,2502
18,Action,1992,1925
19,Action,1993,24623
20,Action,1994,120177
21,Action,1995,216144


___

# 1 PLAYTIMEGENRE

In [9]:
# 1 FUNCION PLAYTIMEGENRE
def PlayTimeGenre(genre: str):

    ### Creo un dataset solo con el listado de datos con el genero brindado
    df_genero = df_playtime_genre[df_playtime_genre['genres'].str.lower() == genre.lower() if isinstance(genre, str) else None]

    ### Si el df está vacio, es porque no encontró informacion para el genero ingreado
    if df_genero.empty:
        return {f"No hay informacion para el genero {genre}": None}

    if not df_genero['playtime_forever'].empty:
        
        max_hours = df_genero.loc[df_genero['playtime_forever'].idxmax(), 'release_year']

        return {f"Año de lanzamiento con mayor horas de juego para el genero {genre}": max_hours}
    else:
        return {f"No hay informacion para el genero {genre}": None}

In [11]:
PlayTimeGenre('Action')

{'Año de lanzamiento con mayor horas de juego para el genero Action': '2012'}

___
# 2 USERFORGENRE

In [13]:
def UserForGenre(genre: str):
    
    genre_df = df_user_for_genre[df_user_for_genre['genres'].str.lower() == genre.lower() if isinstance(genre, str) else None]

    if genre_df.empty:
        return {"Usuario con mas horas jugadas para el genero": None, "Horas jugadas por año": {}}

    user_playtime_sum = genre_df.groupby('user_id')['horas'].sum()

    max_horas = user_playtime_sum.idxmax()

    usuario = genre_df[genre_df['user_id'] == max_horas]

    horas_año = dict(zip(usuario['release_year'], usuario['horas']))

    return {f"Usuario con mas horas jugadas para el genero {genre}": max_horas, "Horas jugadas por año": horas_año}

In [14]:
UserForGenre('2D')

{'Usuario con mas horas jugadas para el genero 2D': '76561198083612270',
 'Horas jugadas por año': {'2012': 411.07}}

In [24]:
df_UsersRecommend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53184 entries, 0 to 53183
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_id             53184 non-null  int64 
 1   app_name            53184 non-null  object
 2   posted              53184 non-null  object
 3   sentiment_analysis  53184 non-null  int64 
 4   recommend           53184 non-null  bool  
dtypes: bool(1), int64(2), object(2)
memory usage: 1.7+ MB


___
# 3 USERSRECOMMEND

In [43]:
def UsersRecommend(year):

    year = str(year)
    
    reviews = df_UsersRecommend[(df_UsersRecommend['posted'] == year) & (df_UsersRecommend['recommend'] == True)]
    
    positive_reviews = reviews[reviews['sentiment_analysis'].isin([1, 2])]
    
    count = positive_reviews['item_id'].value_counts().reset_index()
    count.columns = ['item_id', 'recommendations_count']

    top3 = count.head(3)
    
    result = [{"Posicion {}: {}".format(i+1, row['item_id']): row['recommendations_count']} for i, row in top3.iterrows()]
    
    return result 

In [59]:
df_users_recommend

,item_id,app_name,posted,sentiment_analysis,recommend
0,282010,Carmageddon Max Pack,No data,1,True
1,70,Half-Life,2015,0,True
2,70,Half-Life,2011,0,True
3,70,Half-Life,2014,0,True
4,70,Half-Life,2013,2,True
...,...,...,...,...,...
53179,80,Counter-Strike: Condition Zero,2014,1,False
53180,80,Counter-Strike: Condition Zero,2013,2,True
53181,80,Counter-Strike: Condition Zero,2015,2,True
53182,80,Counter-Strike: Condition Zero,2014,1,True


In [57]:
df_users_recommend = df_UsersRecommend.copy()
'''df_users_recommend['release_year'] = df_users_recommend['release_year'].replace('Invalid Format', pd.NA)

# Convertir los valores de la columna 'posted' a tipo numérico (entero)
df_users_recommend['release_year'] = pd.to_numeric(df_users_recommend['release_year'], errors='coerce')

# Rellenar los valores NaN con 0 y luego convertir a tipo entero
df_users_recommend['release_year'] = df_users_recommend['release_year'].fillna(0).astype(int)

df_users_recommend['release_year'].replace(0, 'No data', inplace=True)'''

"df_users_recommend['release_year'] = df_users_recommend['release_year'].replace('Invalid Format', pd.NA)\n\n# Convertir los valores de la columna 'posted' a tipo numérico (entero)\ndf_users_recommend['release_year'] = pd.to_numeric(df_users_recommend['release_year'], errors='coerce')\n\n# Rellenar los valores NaN con 0 y luego convertir a tipo entero\ndf_users_recommend['release_year'] = df_users_recommend['release_year'].fillna(0).astype(int)\n\ndf_users_recommend['release_year'].replace(0, 'No data', inplace=True)"

In [85]:
df_users_recommend

,item_id,app_name,posted,sentiment_analysis,recommend
0,282010,Carmageddon Max Pack,No data,1,True
1,70,Half-Life,2015,0,True
2,70,Half-Life,2011,0,True
3,70,Half-Life,2014,0,True
4,70,Half-Life,2013,2,True
...,...,...,...,...,...
53179,80,Counter-Strike: Condition Zero,2014,1,False
53180,80,Counter-Strike: Condition Zero,2013,2,True
53181,80,Counter-Strike: Condition Zero,2015,2,True
53182,80,Counter-Strike: Condition Zero,2014,1,True


In [86]:
def UsersRecommend(year):

    # Verificar si hay revisiones para el año dado
    if not df_users_recommend.empty:
        # Filtrar las revisiones para el año dado y recomendaciones positivas/neutrales
        recomendaciones = df_users_recommend[df_users_recommend['posted'] == str(year)]
        
        # Ordenar en orden descendente por la cantidad de recomendaciones
        recomendaciones = recomendaciones.sort_values('sentiment_analysis', ascending=False)
        
        # Crear una única línea de resultado
        resultado = {
            "Puesto 1": recomendaciones.iloc[0]['app_name'],
            "Puesto 2": recomendaciones.iloc[1]['app_name'],
            "Puesto 3": recomendaciones.iloc[2]['app_name']
        }
        
        return resultado

In [87]:
UsersRecommend(2010)

{'Puesto 1': "Garry's Mod",
 'Puesto 2': 'Fallout: New Vegas',
 'Puesto 3': 'eversion'}

___
# 4 USERSNOTRECOMMEND


In [32]:
def UsersNotRecommend(year):
    
    year = str(year)
   

    filtered_reviews = df_UsersRecommend[(df_UsersRecommend['posted'] == year) & (df_UsersRecommend['recommend'] == False)]
    
    negative_reviews = filtered_reviews[filtered_reviews['sentiment_analysis'] == 0]
    
    count = negative_reviews['item_id'].value_counts().reset_index()
    count.columns = ['item_id', 'count']
    
    top3_not_recommendations = count.head(3)
    
    
    result = [{"Posicion {}: {}".format(i+1, row['item_id']): row['count']} for i, row in top3_not_recommendations.iterrows()]
    
    return result

In [33]:
UsersNotRecommend(2011)

[{'Posicion 1: 18700': 2}, {'Posicion 2: 63940': 1}, {'Posicion 3: 33460': 1}]

In [34]:
df_sentiment_analysis.head()

,item_id,release_year,sentiment_analysis
0,282010,1997,1
1,282010,1997,1
2,282010,1997,1
3,70,1998,0
4,70,1998,0


___
# 5 SENTIMENTANALYSIS


In [38]:
def sentiment_analysis(year):

    year = str(year)

    filtered_reviews = df_sentiment_analysis[df_sentiment_analysis['release_year'] == year]
   
    sentiment_counts = filtered_reviews['sentiment_analysis'].value_counts().to_dict()
  
    result = {
        'Negative': sentiment_counts.get(0, 0),
        'Neutral': sentiment_counts.get(1, 0),
        'Positive': sentiment_counts.get(2, 0)
    }
    
    return result

In [39]:
sentiment_analysis(2011)

{'Negative': 1856, 'Neutral': 2160, 'Positive': 5919}